In [3]:
from src import util
import tensorflow as tf
import numpy as np

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
sensor_data_trainval, sequence_ids_trainval, labels_trainval = util.ingest_sensor_data('train.csv', 'train_labels.csv')
sensor_data_test, sequence_ids_test, _ = util.ingest_sensor_data('test.csv')

sensor_data_train, sequence_ids_train, labels_train, sensor_data_val, sequence_ids_val, labels_val = util.validation_split(training_fraction=0.7, sensor_data=sensor_data_trainval, sequence_ids=sequence_ids_trainval, labels=labels_trainval)

(25968, 60, 13) 25968
(12218, 60, 13) 12218


In [ ]:
lstm_dim = 128
dense_dim=128
lstm_dropout=0.15

model = tf.keras.Sequential([
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units=lstm_dim, input_shape=(60,13), dropout=lstm_dropout, recurrent_dropout=lstm_dropout)),
    tf.keras.layers.Dense(dense_dim, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    #tf.keras.layers.Dense(10, activation='relu'),
    #tf.keras.layers.Dropout(0.4),
    
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy',tf.keras.metrics.AUC(name='auc')])


model.fit(util.preprocess_sensor_data(sensor_data_trainval), np.array(labels_trainval), validation_split=0.2, epochs=50)
pred = model.predict(util.preprocess_sensor_data(sensor_data_test))
util.generate_submission(pred, sequence_ids_test, 'lstm-bidirectional')

Epoch 1/50
650/650 [==============================] - 58s 84ms/step - loss: 0.6849 - accuracy: 0.5492 - auc: 0.5673 - val_loss: 0.6815 - val_accuracy: 0.5512 - val_auc: 0.5961
Epoch 2/50
650/650 [==============================] - 69s 106ms/step - loss: 0.6778 - accuracy: 0.5714 - auc: 0.5987 - val_loss: 0.6730 - val_accuracy: 0.5884 - val_auc: 0.6154
Epoch 3/50
650/650 [==============================] - 71s 109ms/step - loss: 0.6708 - accuracy: 0.5882 - auc: 0.6208 - val_loss: 0.6677 - val_accuracy: 0.5884 - val_auc: 0.6589
Epoch 4/50
650/650 [==============================] - 73s 112ms/step - loss: 0.6311 - accuracy: 0.6493 - auc: 0.6974 - val_loss: 0.5744 - val_accuracy: 0.7023 - val_auc: 0.7721
Epoch 5/50
650/650 [==============================] - 73s 112ms/step - loss: 0.5876 - accuracy: 0.6937 - auc: 0.7537 - val_loss: 0.5449 - val_accuracy: 0.7443 - val_auc: 0.8107
Epoch 6/50
650/650 [==============================] - 70s 108ms/step - loss: 0.5530 - accuracy: 0.7198 - auc: 0.7904

In [9]:
util.generate_submission(pred, sequence_ids_test, 'lstm')